In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedLineDocument
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE 


In [2]:
testing = False # Set to True to use small files for quick testing, false to use the entire training data
if (testing):
    filename = 'Data/mergedDataSet_test.csv' #Sample file used for quick testing
    commentsFileName = 'Data/comments_small.csv'
else:
    filename = 'Data/mergedDataSet.csv' #The complete file
    commentsFileName = 'Data/comments.csv'
df = pd.read_csv(filename)
# df = pd.read_csv(filename).iloc[0:10000]

In [117]:
# nltk.download('stopwords') # Download it if not already downloaded
stop = stopwords.words('english')
# stop = open('stop_words.txt','r').read().split()
def RemoveStopWords(row):
    row = row.lower().split() # converting to lower case and splitting
    str1 = ''
    for item in row:
        if item not in stop: #removing stop words
            item = re.sub(r'[^\w\s]','',item) #removing punctutions
            str1 += (item + ' ')
    return str1
target = df['merged_rating'].values #Saving the target variable as a numpy array
df = df['comment_text'].apply(RemoveStopWords)


In [118]:
df.to_csv(commentsFileName,index=False) # Writing the comments to a CSV file to be read by TaggedLineDocument next
documents = TaggedLineDocument(commentsFileName) # Tags each sentence (0,1,2,...)

In [143]:
modelSize = 500 # Will represent each comment with a vector of size 500
modelWindow = 8 
model = Doc2Vec(documents, vector_size=modelSize, window=modelWindow, min_count=1, workers=4, dbow_words=1)
# model.save('model_s500_w8') # Uncomment this to save the model to disk

In [144]:
#Creating a numpy array to keep the data and to be used by a machine learning model as the feature vector
data = np.zeros((len(model.docvecs),len(model.docvecs[0])))
for i in range(len(model.docvecs)):
#     if (i%1000==1):
#         print(i)
    data[i]=model.docvecs[i]

In [7]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_sample(data, target)

dfData = pd.DataFrame(data=X_res,dtype=float)

#dfData = dfData.apply(lambda row : row.values.reshape(1,modelSize).tolist(),axis=1).to_frame()
#dfData[0] = dfData[0].apply(lambda x:x[0])
#dfData = dfData.rename(columns={0: "features"})
dfData['label']=y_res
dfData.to_csv('Data/FeaturizedData.csv')

In [145]:
#Creating validation sets
X_train, X_val, y_train, y_val = train_test_split(data,target, test_size=0.2, random_state=0)

In [147]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_sample(X_train, y_train)

In [148]:
def imbalances(y):
    count_0 = 0
    count_1 = 0
    count_2 = 0
    for i in y:
        if(i==0):
            count_0 +=1
        elif(i==1):
            count_1 +=1
        else:
            count_2 +=1
    print('0:',count_0)
    print('1:',count_1)
    print('2:',count_2)

imbalances(y_train)
imbalances(y_res)

0: 115281
1: 6274
2: 6936
0: 115281
1: 115281
2: 115281


In [178]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
from imblearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron

In [ ]:
print('started')
clf = GaussianNB()
# clf = Perceptron(penalty='l2',class_weight='balanced',max_iter=100)
clf = RandomForestClassifier(random_state=0,n_estimators=50,max_depth=20)
#clf = SVC(class_weight='balanced')
pipeline = make_pipeline(clf)
pipline_model = pipeline.fit(X_train, y_train)


started


In [ ]:
y_pred = pipline_model.predict(X_val)
print("Number of mislabeled points out of a total %d points : %d"%(y_val.shape[0],(y_val != y_pred).sum()))

In [212]:
precision_recall_fscore_support(y_val,y_pred,average='weighted')

(0.87702426807432809, 0.88821093920244065, 0.87476638343445634, None)